In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3 as sql
sns.set()
%matplotlib inline

In [2]:
conn = sql.connect('data/test.db')

In [3]:
np.random.seed(1897)
df_1 = pd.DataFrame({'Id':np.arange(1,21),
                     'ProdId': ['A','B','C','D']*5,
                     'Length': np.random.randint(20,40,size=20),
                     'Height': np.random.randint(10,35,size=20)})

df_2 = pd.DataFrame({'Id':np.random.randint(10,size=100),
                     'Date':pd.to_timedelta(np.random.randint(0,1000,size=100), unit='d')+pd.datetime(2015,1,1),
                     'Cat_1': np.array(['A','B','C','D'])[np.random.randint(4,size=100)],
                     'Cat_2': np.array(['S','T','Q','R'])[np.random.randint(4,size=100)]})

# Assignment 1
Load the sheets `Customers` and `Products` from the Excel file `data/master_data.xlsx`. Store the data from the `Customers` sheet in `cust_df` and the data from the `Products` sheet in `prod_df`.
<details><summary>Hint 1</summary>
    <p>
        The pandas function `read_excel` has an option `sheet_name`, that let's one specify which sheets to extract
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        The pandas function `read_excel` returns a dictionary with the sheet names as keys
    </p>
</details>

In [4]:
cust_df = pd.read_excel('data/master_data.xlsx','Customers')
prod_df = pd.read_excel('data/master_data.xlsx','Products')

# Assignment 2
The CSV file `data/bad_form.csv` follows a rather odd format:
* The columns are separated by `|`
* There are several comment lines. Each comment is indicated by `//`
* True values in the Boolean columns are marked by `Yes`
* False values in the Boolean columns are marked by `Nope`
* Decimals are indicated by `,`
* The string columns contain characters in `latin-1` encoding
* In column **B** the missing values are indicated by `Missing`
* In column **C** the missing values are indicated by `Not here`
* The first 5 lines in the file is merely a header - should not be loaded
* The last 10 lines make out the footer - should not be loaded

Load the file in a DataFrame with the column `Id` as index.
<details><summary>Hint 1</summary>
    <p>
        Look into the documentation for parameters `sep`, `comment`, `true_values`, `false_values`, `index_col`, `skiprows`, `skipfooter`, `na_values`, `encoding` and `decimal` for the pandas function `read_csv`.
    </p>
</details>

In [5]:
bad_form_df = pd.read_csv('data/bad_form.csv',engine = 'python',sep ='|',comment = '//',true_values = ['Yes'],false_values = ['Nope'],decimal = ',',encoding = 'latin-1',na_values = ['Missing','Not here'],skiprows = 5,skipfooter = 10,index_col = 0)
bad_form_df.head(10)

A      B       C         D       E
Id                                        
1    True    NaN  Whisky  2.500000    Else
2    True  Hotel     NaN  0.400000  Elsker
3   False   Echo   Oskar  1.200000   Pelse
4   False    NaN     NaN  8.000000    Else
7   False    NaN   Romeo  4.700000  Elsker
8    True    NaN    Lima  3.141592   Pølse
9   False   Lima     NaN  2.717270  Pølser
11   True    NaN     NaN  0.690000      Åd
12  False   Lima   Delta  1.410000     Hun
13  False    NaN       !  1.618000   Dagen

# Assignment 3
Write the content of the DataFrame `df_1` into the table `product_dimensions` in the database `test.db`. The connection to the database has already been set up in the variable `conn`.
<details><summary>Hint 1</summary>
    <p>
        Use the `to_sql` method for the DataFrame to write it to the database
    </p>
</details>

In [6]:
df_1.to_sql(name='product_dimensions',con = conn, if_exists='replace' )

# Assignment 4
Compute the average area of each product in `product_dimensions` using a SQL statement. Then proceed to load the data in two lines at a time, and print out the result.
<details><summary>Hint 1</summary>
    <p>
        The pandas function `read_sql` can be used to parse a SQL statement to a database and store the result into a DataFrame
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Setting the parameter `chunksize` controls the number of records returned at a time
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        With a set `chunksize` you can subsequently loop over the reader object, that is returned from `read_sql`
    </p>
</details>

In [7]:
for chunk in pd.read_sql('select product_dimensions.ProdId, AVG(product_dimensions.Length) AS length,AVG(product_dimensions.Height) AS height from product_dimensions group by product_dimensions.ProdId',con=conn,chunksize = 2):
    print(chunk)

  ProdId  length  height
0      A    31.0    20.6
1      B    29.0    20.2
  ProdId  length  height
0      C    26.4    26.2
1      D    28.2    23.8


# Assignment 5
For each `ID` extract the records up until the first time a `B` or a `C` appears in the `Cat_1` column, or a `R` or a `S` appears in the `Cat_2` column. The `Date` column indicates the order of the observations.
<details><summary>Hint 1</summary>
    <p>
        The `groupby` method preserves the order of the records as they had in the original DataFrame
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Applying the `cummax` method to a Boolean column returns 0 for every record, until the first True value is reached. From that point on it returns a 1.
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        Using the index to merge the results onto the original DataFrame makes it possible to select the rows with value 0 in the new column.
    </p>
</details>

In [23]:
#création des dictonnaires (utilisés pour savoir si on prend la ligne ou pas)
dict_Cat1= {'B':True,'C':True}
dict_Cat2= {'R':True,'S':True}
list_id= {0,1,2,3,4,5,6,7,8,9}
#création autre dataframe pour rajouter les colonnes Bool_Cat_1 et bool_cat_2
df_test= pd.DataFrame(data=df_2)


In [9]:
#on rajoute les colonnes Bool_Cat_1 et bool_Cat_2 et on fait les tests
df_test['Bool_Cat_1'] = df_2['Cat_1'].map(dict_Cat1)
df_test['Bool_Cat_2'] = df_2['Cat_2'].map(dict_Cat2)

In [61]:
#toute valeur en NaN passe à false.
df_test['Bool_Cat_1'] = df_test['Bool_Cat_1'].fillna(value = False)
df_test['Bool_Cat_2'] = df_test['Bool_Cat_2'].fillna(value = False)
df_sort= pd.DataFrame(columns=df_test.columns)



In [62]:
ID = 0

while ID <= 9:
    Index = 0
    # on recupère les rows correspondant à l'ID
    df_ID = df_test.loc[df_test['Id'] == ID]
    # on reset l'id
    df_ID=df_ID.reset_index()
    del df_ID['index']
    #on récupère la valeur max de l'id
    max_index= df_ID.last_valid_index()
    while Index <= max_index:
        if df_ID.at[Index,'Bool_Cat_1'] == True or df_ID.at[Index,'Bool_Cat_2'] == True  :
            Index = max_index + 1
            ID = ID + 1
        else:
            df_sort = df_sort.append(df_ID.loc[[Index]],ignore_index=True)
            Index = Index + 1
           

    


In [63]:
df_sort

Id       Date Cat_1 Cat_2 Bool_Cat_1 Bool_Cat_2
0  0 2017-08-28     A     Q      False      False
1  0 2016-06-07     D     T      False      False
2  0 2016-11-20     D     T      False      False
3  3 2016-04-11     A     Q      False      False
4  8 2017-05-23     A     T      False      False